## Intorduction

This notebook shows how to to segment and cluster the neighborhood after we have scraped the wikipedia page to get the data of Toronto neighborhood.We will use k-means clustering algorithm. We will also use Folium library to visualize the neighborhoods and their emerging clusters.

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  an

In [2]:
conda install -c anaconda lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.3.0-py36hefd8a0e_0 anaconda


lxml-4.3.0           | 1.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following pa

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [4]:
conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

## 1.Access the Dataset and analyze it

In [2]:
page = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
page

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XjZTxApAAD0AAJXYUvgAAAFV","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in 

In [3]:
soup = bs(page)
soup.prettify()
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))
nbrs= df[0] # get the first table
nbrs = nbrs[nbrs.Borough != 'Not assigned']
for index, row in nbrs.iterrows():
    if row['Neighbourhood'] =='Not assigned':
        row ['Neighbourhood'] = row['Borough'] 

nbrs = nbrs.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
nbrs

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


**Get the latitude and longitude of each postal code for Toronto**

In [7]:
gs= pd.read_csv("http://cocl.us/Geospatial_data")
gs.columns = ['Postcode', 'Latitude', 'Longitude']
gs.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Join the latitude and longitude of each postal code with toronto neighborhood data**

In [8]:
nbrs = pd.merge(nbrs, gs, on='Postcode', how='outer', validate="one_to_one")
nbrs.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Use geopy library to get the latitude and longitude values of Toronto City**

In [9]:
# convert an address into latitude and longitude values
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


**Use folium library to plot aLL Neighbourhood of Toronto City**

In [10]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

In [11]:

# add markers to map
for index, row in nbrs.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    neighborhood = row['Neighbourhood']
    borough = row['Borough']
    label = '{}, {}'.format(neighborhood, borough).replace("'", "")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)

map_toronto

## 2.Explore the Neighborhood of Downtown Toronto Area

In [12]:
downtown_data = nbrs[nbrs['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


**Plot neighborhood in DownTown Toronto**

In [13]:
address = 'Downtown Toronto, TORONTO'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [14]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

map_downtown

In [16]:
# add markers to map
for index, row in downtown_data.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    neighborhood = row['Neighbourhood']
    borough = row['Borough']
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)

map_downtown

**Use Foursquare API to get the venues in Downtown Toronto**

In [17]:
CLIENT_ID = 'D3SLYZQDESMDQVNIM25V4HATKCRSUFZWDBWAYSSFSEAQBR4A' # Removed to publish on GitHub
CLIENT_SECRET = 'OFAIJIZG4200P5J1MPJOYDHZSVUHJYMRBNCJN1Y3FWMFH3ZL' # Removed to publish on GitHub
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D3SLYZQDESMDQVNIM25V4HATKCRSUFZWDBWAYSSFSEAQBR4A
CLIENT_SECRET:OFAIJIZG4200P5J1MPJOYDHZSVUHJYMRBNCJN1Y3FWMFH3ZL


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url, verify=False).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Cabbagetown,St. James Town
Church and Wellesley


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Harbourfront


...

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Christie


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Queen's Park


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [20]:
downtown_venues
downtown_venues.rename(columns={'Neighborhood': 'Neighbourhood'}, inplace = True)
downtown_venues

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
...,...,...,...,...,...,...,...
1295,Queen's Park,43.662301,-79.389494,Tim Hortons,43.658175,-79.390681,Coffee Shop
1296,Queen's Park,43.662301,-79.389494,SUDS,43.659880,-79.394712,Bar
1297,Queen's Park,43.662301,-79.389494,Tim Hortons,43.658599,-79.388498,Coffee Shop
1298,Queen's Park,43.662301,-79.389494,The Green Beet Cafe,43.662096,-79.394153,Vegetarian / Vegan Restaurant


In [21]:
downtown_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
"Chinatown,Grange Park,Kensington Market",86,86,86,86,86,86
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 197 uniques categories.


## 3. Analyze Each Neighborhood in Downtown Toronto

In [23]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"Cabbagetown,St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.010000,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.00,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000
3,"Cabbagetown,St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.0,0.0,...,0.0,0.0,0.0,0.0,0.012195,0.0,0.0,0.012195,0.00,0.012195


**Print each neighborhood along with the top 5 most common venues**

In [25]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1        Steakhouse  0.04
2              Café  0.04
3        Restaurant  0.03
4  Asian Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2          Bakery  0.04
3  Farmers Market  0.04
4        Beer Bar  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0     Airport Service  0.21
1      Airport Lounge  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07


----Cabbagetown,St. James Town----
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.07
2  Italian Restaurant  0.05
3                Café  0.05
4                 Pub  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3                Café 

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 11

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
   
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

downtown_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Café,Restaurant,Bar,Bakery,Cosmetics Shop,Thai Restaurant,Asian Restaurant,Burger Joint,Bookstore
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Concert Hall,Shopping Mall
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Sculpture Garden,Airport,Airport Food Court,Comic Shop
3,"Cabbagetown,St. James Town",Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza,Playground
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place,Dessert Shop


## 4. Cluster Neighborhoods in Downtown Toronto

**Run k-means to cluster the neighborhood into 5 clusters.**

In [28]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 2, 0, 3, 0, 4, 0, 0, 0, 0, 0, 3, 3, 3, 1, 0, 0, 0],
      dtype=int32)

**create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood**

In [29]:
# add clustering labels

downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
downtown_merged =downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
downtown_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza,Playground
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Restaurant,Fast Food Restaurant,Café,Grocery Store,Gym,Hotel,Gastropub
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Farmers Market,Performing Arts Venue,Ice Cream Shop,Hotel
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Italian Restaurant,Pizza Place,Bubble Tea Shop,Electronics Store,Bakery,Middle Eastern Restaurant


**Visualize the resulting clusters**

In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

**Cluster 1**

In [31]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza,Playground
2,Downtown Toronto,0,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Restaurant,Fast Food Restaurant,Café,Grocery Store,Gym,Hotel,Gastropub
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Italian Restaurant,Pizza Place,Bubble Tea Shop,Electronics Store,Bakery,Middle Eastern Restaurant
5,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Breakfast Spot,Cosmetics Shop,Hotel,Beer Bar,American Restaurant,Bakery,Cocktail Bar,Clothing Store
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Concert Hall,Shopping Mall
8,Downtown Toronto,0,Coffee Shop,Steakhouse,Café,Restaurant,Bar,Bakery,Cosmetics Shop,Thai Restaurant,Asian Restaurant,Burger Joint,Bookstore
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Steakhouse,Restaurant,Italian Restaurant,Bar,Gastropub,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
11,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Bakery,Seafood Restaurant,Steakhouse,Italian Restaurant,Deli / Bodega,American Restaurant
12,Downtown Toronto,0,Café,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Nightclub,Noodle House,Coffee Shop,Chinese Restaurant,Dessert Shop
13,Downtown Toronto,0,Vietnamese Restaurant,Café,Chinese Restaurant,Bar,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Burger Joint,Donut Shop,Grocery Store


**Cluster 2**

In [32]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


**Cluster 3**

In [33]:

downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Sculpture Garden,Airport,Airport Food Court,Comic Shop


**Cluster 4**

In [34]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
3,Downtown Toronto,3,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Farmers Market,Performing Arts Venue,Ice Cream Shop,Hotel
7,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place,Dessert Shop
9,Downtown Toronto,3,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Pizza Place,Park
18,Downtown Toronto,3,Coffee Shop,Park,Gym,Burger Joint,Salad Place,Portuguese Restaurant,Sandwich Place,Juice Bar,Chinese Restaurant,Burrito Place,Italian Restaurant


**CLuster 5**

In [35]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Nightclub,Coffee Shop,Restaurant,Gas Station,Baby Store,Italian Restaurant,Diner,Candy Store
